# TASK DESCRIPTION

**Legend:**

Young Alex has a beloved BERT model that he carries everywhere on his trusty flash drive. One day, during an excursion along the River Styx, a few drops of water landed on the precious device, corrupting the model's weights.

Heartbroken, Alex rushed home to fix the neural network. After quick analysis, he discovered only the token embeddings were damaged - the rest of the architecture (attention blocks and heads) remained perfectly intact. Now he needs to restore the model's performance on Sentiment Analysis Task.

**Task:**

You need to fix the broken vectors of the Embeddings matrix of the model so as to improve the quality of the model on the task of text sentiment analysis.

**Restrictions:**

- You can not use any other transformer based pre-trained models and LLMs.

- You can not any additional data

- You can not fine-tune or pre-train model

===

When you make a submit, make a Quick Save of the notebook, otherwise we may reject your solution.

You must solve this task on KAGGLE (YOU CAN'T USE CLOUD.RU)

==========

**Легенда:**

Young Alex имеет любимую модель BERT, которую он везде носит на своей надежной флешке. Однажды, во время экскурсии вдоль реки Стикс, несколько капель воды попало на драгоценное устройство, повредив веса модели.

С разбитым сердцем Алекс поспешил домой, чтобы починить нейронную сеть. После быстрого анализа он обнаружил, что повреждены только эмбеддинги токенов — остальная архитектура (блоки внимания и головы) осталась полностью нетронутой.

Теперь ему нужно восстановить производительность модели, оставив все остальные веса замороженными (никакие изменения в механизмах внимания или других компонентах не допускаются). Ваша задача — помочь Алексу достичь этой цели, не нарушив его ностальгическую привязанность к оригинальной модели.

**Задача:**

Вам необходимо починить сломанные вектора матрицы Embeddings модели так, чтобы улучшить качество модели на задаче анализа тональности текста.

**Ограничения:**

- Вы не можете использовать никакие другие предобученные модели на основе архитектуры Трансформер и LLM.

- Вы не можете использовать никакие дополнительные данные.

- Вы не можете дообучать или предобучать модель.

===

При отправке решения сделайте Quick Save ноутбука, иначе мы можем отклонить ваше решение.

Эту задачу необходимо решить на KAGGLE (ВЫ НЕ МОЖЕТЕ ИСПОЛЬЗОВАТЬ CLOUD.RU)

# DEPENDINGS

In [ ]:
import numpy as np
import pandas as pd
import torch
np.random.seed(21)

# LOAD DATASET

In [ ]:
val_data_path = "/kaggle/input/neoai-2025-broken-bert/val_dataset.csv"
test_data_path = "/kaggle/input/neoai-2025-broken-bert/test.csv"

val_df = pd.read_csv(val_data_path)

test_df = pd.read_csv(test_data_path)

# LOAD TOKENIZER & MODEL

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Ilseyar-kfu/broken_bert")

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
val_encodings = tokenizer(val_df["text"].to_list(), truncation=True, padding=True, max_length=256)
val_dataset = Dataset(val_encodings, val_df["labels"].to_list())

In [ ]:
texts_2_score = val_df["text"].to_list() + test_df["text"].to_list()

# MODEL CHANGES

In [27]:
model = AutoModelForSequenceClassification.from_pretrained("Ilseyar-kfu/broken_bert")

new_embedings = model.bert.embeddings.word_embeddings.weight.detach().numpy().copy()

# There's magic going on here!!! And we get very new !!! new_embedings !!!

model.bert.embeddings.word_embeddings.weight = torch.nn.Parameter(torch.Tensor(new_embedings))

In [28]:
new_embedings.shape

(30522, 768)

In [78]:
from sklearn.decomposition import PCA
import torch
import numpy as np

# Load the pre-trained model with corrupted embeddings
model = AutoModelForSequenceClassification.from_pretrained("Ilseyar-kfu/broken_bert")

# --- Start of the fix ---

# 1. Extract the embedding matrix and its dimensions
embeddings = model.bert.embeddings.word_embeddings.weight.detach().cpu().numpy()
embedding_dim = embeddings.shape[1]

# 2. Reset the [PAD] token's embedding to a zero vector for stability
pad_token_id = tokenizer.pad_token_id
if pad_token_id is not None:
    embeddings[pad_token_id] = np.zeros(embedding_dim)

# 3. Center the embedding data by subtracting the mean
mean_vector = np.mean(embeddings, axis=0)
centered_embeddings = embeddings - mean_vector

# 4. Apply PCA to de-noise the embeddings.
# We retain the components that explain 99% of the variance.
pca = PCA(n_components=0.99)
transformed_embeddings = pca.fit_transform(centered_embeddings)

# 5. Reconstruct the embeddings from the reduced PCA space
reconstructed_centered_embeddings = pca.inverse_transform(transformed_embeddings)

# 6. Add the mean vector back to the reconstructed embeddings
reconstructed_embeddings = reconstructed_centered_embeddings + mean_vector

# The 'magic' is the de-noised embedding matrix
new_embedings = reconstructed_embeddings

# --- End of the fix ---

# 7. Load the repaired embeddings back into the model
model.bert.embeddings.word_embeddings.weight = torch.nn.Parameter(torch.Tensor(new_embedings))

In [84]:
# %% [code]
import numpy as np
import torch

# Загружаем модель
model = AutoModelForSequenceClassification.from_pretrained("Ilseyar-kfu/broken_bert")

# --- Начало нового метода исправления ---

# 1. Извлекаем эмбеддинги
embeddings = model.bert.embeddings.word_embeddings.weight.detach().cpu().numpy().copy()

# 2. Для каждого вектора вычисляем его длину (L2-норму)
norms = np.linalg.norm(embeddings, axis=1)

# 3. Находим "выбросы" по нормам. Используем перцентили для определения "нормального" диапазона.
# Это более надежно, чем среднее и стандартное отклонение.
p_low = np.percentile(norms, 1)  # Нижняя граница нормы (1-й перцентиль)
p_high = np.percentile(norms, 99) # Верхняя граница нормы (99-й перцентиль)

# Находим индексы векторов, которые выходят за пределы "нормального" диапазона
outlier_indices = np.where((norms < p_low) | (norms > p_high))[0]
good_indices = np.where((norms >= p_low) & (norms <= p_high))[0]

print(f"Найдено {len(outlier_indices)} предположительно поврежденных векторов.")

# 4. Вычисляем средний вектор на основе всех "хороших" эмбеддингов
if len(good_indices) > 0:
    # Вычисляем среднее значение только по "хорошим" векторам
    mean_good_embedding = np.mean(embeddings[good_indices], axis=0)

    # 5. Заменяем все аномальные векторы на вычисленный средний вектор
    if len(outlier_indices) > 0:
        embeddings[outlier_indices] = mean_good_embedding
        print("Поврежденные векторы заменены на среднее значение 'хороших' векторов.")
else:
    print("Не удалось найти 'хорошие' векторы. Эмбеддинги не изменены.")

# Наша "магия" — это исправленная матрица эмбеддингов
new_embedings = embeddings

# --- Конец нового метода исправления ---

# Загружаем исправленные эмбеддинги обратно в модель
model.bert.embeddings.word_embeddings.weight = torch.nn.Parameter(torch.Tensor(new_embedings))

Найдено 306 предположительно поврежденных векторов.
Поврежденные векторы заменены на среднее значение 'хороших' векторов.


In [ ]:
# %% [markdown]
# # FIX EMBEDDINGS VIA WORD2VEC TRAINING ON PROVIDED TEXTS
# Train non-transformer Word2Vec embeddings on the val+test corpus,
# then map them into the BERT embedding matrix.

# %% [code]
from gensim.models import Word2Vec
from transformers import AutoModelForSequenceClassification
import numpy as np
import torch

# collect token-level corpus using BERT tokenizer
corpus = [tokenizer.tokenize(text) for text in texts_2_score]

# train Word2Vec with small vector size matching BERT hidden size
wv_size = model.config.hidden_size  # typically 768
w2v = Word2Vec(corpus, vector_size=wv_size, window=5, min_count=1, workers=4, epochs=10)

# build new embedding matrix
vocab = tokenizer.get_vocab()
vocab_size = model.config.vocab_size

ew_matrix = np.zeros((vocab_size, wv_size), dtype=np.float32)
avg_vec = np.mean(w2v.wv.vectors, axis=0)

for token, idx in vocab.items():
    if token in w2v.wv:
        ew = w2v.wv[token]
    else:
        ew = avg_vec
    ew_matrix[idx] = ew

# assign to model
model = AutoModelForSequenceClassification.from_pretrained("Ilseyar-kfu/broken_bert")
model.bert.embeddings.word_embeddings.weight = torch.nn.Parameter(torch.tensor(ew_matrix))

# freeze everything except embeddings (optional)
for name, param in model.named_parameters():
    if 'embeddings.word_embeddings' not in name:
        param.requires_grad = False

# %% [markdown]
# Now evaluate on validation set as before:
# %% [code]
evaluate_on_validation(model, tokenizer, val_df)

# EVALUATION

In [88]:
from sklearn.metrics import f1_score
from numpy import argmax
from transformers import pipeline
import wandb
wandb.init(mode= "disabled")

In [89]:
from sklearn.metrics import classification_report

def evaluate_on_validation(model, tokenizer, df_val):
    label_2_dict = {'LABEL_0': 'neutral', "LABEL_1" : 'positive', "LABEL_2": 'negative'}
    classifier = pipeline("text-classification", model= model, tokenizer = tokenizer)
    answ = classifier.predict(list(df_val["text"]))
    answ = [label_2_dict[el["label"]] for el in answ]
    
    # print(f1_score(p.label_ids, preds, average='macro'))
    print(classification_report(df_val["labels"], answ))

In [87]:
evaluate_on_validation(model, tokenizer, val_df)

Device set to use cuda:0


              precision    recall  f1-score   support

    negative       0.60      0.18      0.27       935
     neutral       0.32      0.91      0.47       759
    positive       0.62      0.06      0.11       806

    accuracy                           0.36      2500
   macro avg       0.52      0.38      0.29      2500
weighted avg       0.52      0.36      0.28      2500



# MODEL SCORING
When you make a submit, 
1. Make a Quick Save of the notebook, otherwise we may reject your solution! 
2. Add notebook version to the comment for the submit.

===

При отправке решения:

1. Сделайте Quick Save ноутбука, иначе мы можем отклонить ваше решение!
2. Добавьте версию ноутбука в комментарий к отправке.

In [82]:
import hashlib

def create_submission(model, tokenizer, df_test):
    label_2_dict = {'LABEL_0': 'neutral', "LABEL_1" : 'positive', "LABEL_2": 'negative'}
    classifier = pipeline("text-classification", model= model, tokenizer = tokenizer)
    answ = classifier.predict(list(df_test["text"]))
    answ = [label_2_dict[el["label"]] for el in answ]
    
    df = pd.DataFrame({"labels" : answ, "id": df_test['id']})
    hsh = hashlib.sha256(df.to_csv(index=False).encode('utf-8')).hexdigest()[:8]
    submit_path = f"submit_{hsh}.csv"
    print(f"SUBMIT_NAME: {submit_path}")
    print(df.head(10))
    df.to_csv(submit_path,index=False)

In [83]:
create_submission(model, tokenizer, test_df)

Device set to use cuda:0


SUBMIT_NAME: submit_e5690a38.csv
     labels    id
0   neutral  5000
1   neutral  5001
2   neutral  5002
3   neutral  5003
4   neutral  5004
5   neutral  5005
6   neutral  5006
7  negative  5007
8   neutral  5008
9   neutral  5009
